In [2]:
import time as t
import datetime as dt
import pandas as pd
import re
import os
import sys
sys.path.insert(1, '../')
from utils.common import *
load_dotenv()
from solana.rpc.websocket_api import connect
from solders.pubkey import Pubkey
from solders.rpc.config import (
    RpcAccountInfoConfig,
    RpcProgramAccountsConfig,
    RpcSignatureSubscribeConfig,
    RpcTransactionLogsConfig,
    RpcTransactionLogsFilter,
    RpcTransactionLogsFilterMentions,
)


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.float_format', lambda x: '%.10f' % x)

japikey = os.getenv('bi_api_key')
uri = 'websocketuri' ## needs to be hosted


In [40]:
pr_string = '675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8'
program = Pubkey.from_string(pr_string)
account = Pubkey.from_string("7YttLkHDoNj9wyDur5pM1ejNaAvT9X4eqaYcHQqtj2G5")

In [8]:
websocket = await connect(uri)

In [10]:
RpcTransactionLogsFilterMentions.pubkey = account

In [13]:
async with connect(uri) as websocket:
    print("Connected to the Solana RPC WebSocket.")
    
    # Subscribe to the program
    await websocket.logs_subscribe(filter_= RpcTransactionLogsFilterMentions(account),commitment='finalized')
    
    message = await websocket.recv()
    print("Subscription: ", message)
    subscription_id = message[0].result

    # Now, enter a loop to continuously receive messages
    try:
        for i in range(3):
            message = await websocket.recv()  # Use recv to get the next message
            sig = re.findall(r"signature: \"([^\"]+)\"", str(message[0]))[0]
            print("Raydium in: ", pr_string in str(message[0]), "Signature: ",sig)
        await websocket.logs_unsubscribe(subscription_id)
    except KeyboardInterrupt:
        print("Subscription ended.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        print('unsubing')
        await websocket.logs_unsubscribe(subscription = int(subscription_id))

Connected to the Solana RPC WebSocket.
Subscription:  [SubscriptionResult {
    jsonrpc: TwoPointOh,
    id: 1,
    result: 404378,
}]
Received message:  [LogsNotification {
    result: LogsNotificationResult {
        context: RpcResponseContext {
            slot: 254719040,
            api_version: None,
        },
        value: RpcLogsResponse(
            RpcLogsResponse {
                signature: "2Gwinf7x5ohCPxFXU4tfhNgL1Xk6kZpmj1EYXVK1k7sbaRzLgmUkHmSE6gAGyqiKesmGRPcV2WJ6TjG1yz7LJDaE",
                err: None,
                logs: [
                    "Program ComputeBudget111111111111111111111111111111 invoke [1]",
                    "Program ComputeBudget111111111111111111111111111111 success",
                    "Program ComputeBudget111111111111111111111111111111 invoke [1]",
                    "Program ComputeBudget111111111111111111111111111111 success",
                    "Program 11111111111111111111111111111111 invoke [1]",
                    "Program 111111

KeyError: 404378

In [14]:
websocket.messages

deque(['{"jsonrpc":"2.0","result":true,"id":2}'])

In [ ]:
await websocket.unsubscribe()
await websocket.close()